In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb
%matplotlib inline
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import warnings
from collections import Counter
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
warnings.filterwarnings("ignore")

In [2]:
trainFeatures = pd.read_csv('./input/FinalTrainFeatures.csv')
testFeatures = pd.read_csv('./input/FinalTestFeatures.csv')
age_train = pd.read_csv('./input/age_train.csv', header=None)

In [3]:
age_train.columns=['uId', 'age_group']
trainLabels = age_train.age_group

In [4]:
X = trainFeatures.drop(['uId'], axis=1)
y = trainLabels
uid = testFeatures['uId']
test = testFeatures.drop('uId', axis=1)

In [11]:
import xgboost as xgb
#train xgb
config = {
    'rounds': 10000,
    'folds': 5
}

params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 7,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 12                  # cpu 线程数
}

''''gpu_id': 0,
    'tree_method': 'gpu_hist',
    'max_bin': 16
    '''

def xgbCV(trainFeature, trainLabel, params, rounds):
    dtrain = xgb.DMatrix(trainFeature, label = trainLabel)
    params['scale_pos_weights '] = (float)(len(trainLabel[trainLabel == 0]))/(float)(len(trainLabel[trainLabel == 1]))
    num_round =rounds
    print ('run cv: ' + 'round: ' + str(rounds))
    res = xgb.cv(params, dtrain, num_round, verbose_eval = 100, early_stopping_rounds=200, nfold=3)
    return res

def xgbPredict(trainFeature,trainLabel,testFeature,rounds,params):
    params['scale_pos_weights '] = (float)(len(trainLabel[trainLabel == 0]))/len(trainLabel[trainLabel == 1])
    
    dtrain = xgb.DMatrix(trainFeature.values, label = trainLabel)
    dtest = xgb.DMatrix(testFeature.values)

    watchlist  = [(dtrain,'train')]
    num_round = rounds
    
    model = xgb.train(params, dtrain, num_round, watchlist, verbose_eval = 50)
    predict = model.predict(dtest)
    return model,predict

In [ ]:
cv_res = xgbCV(X, y, params, 10000)
cv_res['test-SCORE-mean'][-1:]

run cv: round: 10000
[0]	train-merror:0.537696+0.000307366	test-merror:0.5681+0.000401029


In [ ]:
model, predict = xgbPredict(trainFeature,trainLabel,testFeature,700,params)

In [ ]:
result = pd.DataFrame({'id':uid, 'label':predict})
result.to_csv("NewResul54.csv", index=False)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(trainFeatures, trainLabels, test_size=0.1)

In [10]:
param = {
        'learning_rate': 0.1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 20,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 230,
        'feature_fraction': 0.8,
        'metric': 'multi_error'
        }

X = trainFeatures.drop(['uId'], axis=1)
y = trainLabels
uid = testFeatures['uId']
test = testFeatures.drop('uId', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=3, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=10000, valid_sets=[vali], early_stopping_rounds=500)
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    y_test = model.predict(test,num_iteration=model.best_iteration)
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission-1.csv',index=False)

0
training start...
[1]	valid_0's multi_error: 0.701493
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's multi_error: 0.692427
[3]	valid_0's multi_error: 0.666593
[4]	valid_0's multi_error: 0.653592
[5]	valid_0's multi_error: 0.639051
[6]	valid_0's multi_error: 0.622156
[7]	valid_0's multi_error: 0.608626
[8]	valid_0's multi_error: 0.597448
[9]	valid_0's multi_error: 0.587695
[10]	valid_0's multi_error: 0.580089
[11]	valid_0's multi_error: 0.574272
[12]	valid_0's multi_error: 0.569786
[13]	valid_0's multi_error: 0.566025
[14]	valid_0's multi_error: 0.562348
[15]	valid_0's multi_error: 0.559605
[16]	valid_0's multi_error: 0.55742
[17]	valid_0's multi_error: 0.555396
[18]	valid_0's multi_error: 0.553265
[19]	valid_0's multi_error: 0.551454
[20]	valid_0's multi_error: 0.549868
[21]	valid_0's multi_error: 0.548292
[22]	valid_0's multi_error: 0.546953
[23]	valid_0's multi_error: 0.545931
[24]	valid_0's multi_error: 0.544592
[25]	valid_0's multi_error: 0.543435
[2

[218]	valid_0's multi_error: 0.484278
[219]	valid_0's multi_error: 0.484228
[220]	valid_0's multi_error: 0.484196
[221]	valid_0's multi_error: 0.484108
[222]	valid_0's multi_error: 0.483957
[223]	valid_0's multi_error: 0.483923
[224]	valid_0's multi_error: 0.483835
[225]	valid_0's multi_error: 0.483737
[226]	valid_0's multi_error: 0.483666
[227]	valid_0's multi_error: 0.483656
[228]	valid_0's multi_error: 0.483528
[229]	valid_0's multi_error: 0.483459
[230]	valid_0's multi_error: 0.483286
[231]	valid_0's multi_error: 0.483255
[232]	valid_0's multi_error: 0.483208
[233]	valid_0's multi_error: 0.48315
[234]	valid_0's multi_error: 0.483078
[235]	valid_0's multi_error: 0.482952
[236]	valid_0's multi_error: 0.482928
[237]	valid_0's multi_error: 0.482917
[238]	valid_0's multi_error: 0.48282
[239]	valid_0's multi_error: 0.482743
[240]	valid_0's multi_error: 0.482616
[241]	valid_0's multi_error: 0.482643
[242]	valid_0's multi_error: 0.482557
[243]	valid_0's multi_error: 0.482605
[244]	valid_0'

[435]	valid_0's multi_error: 0.473944
[436]	valid_0's multi_error: 0.473914
[437]	valid_0's multi_error: 0.473869
[438]	valid_0's multi_error: 0.473814
[439]	valid_0's multi_error: 0.473784
[440]	valid_0's multi_error: 0.473717
[441]	valid_0's multi_error: 0.473713
[442]	valid_0's multi_error: 0.473662
[443]	valid_0's multi_error: 0.473626
[444]	valid_0's multi_error: 0.473658
[445]	valid_0's multi_error: 0.473735
[446]	valid_0's multi_error: 0.473683
[447]	valid_0's multi_error: 0.473643
[448]	valid_0's multi_error: 0.473593
[449]	valid_0's multi_error: 0.473592
[450]	valid_0's multi_error: 0.473537
[451]	valid_0's multi_error: 0.473531
[452]	valid_0's multi_error: 0.473525
[453]	valid_0's multi_error: 0.473489
[454]	valid_0's multi_error: 0.473513
[455]	valid_0's multi_error: 0.473471
[456]	valid_0's multi_error: 0.473422
[457]	valid_0's multi_error: 0.473395
[458]	valid_0's multi_error: 0.47349
[459]	valid_0's multi_error: 0.473458
[460]	valid_0's multi_error: 0.473456
[461]	valid_0

[652]	valid_0's multi_error: 0.470653
[653]	valid_0's multi_error: 0.470647
[654]	valid_0's multi_error: 0.470638
[655]	valid_0's multi_error: 0.470672
[656]	valid_0's multi_error: 0.470652
[657]	valid_0's multi_error: 0.470628
[658]	valid_0's multi_error: 0.470631
[659]	valid_0's multi_error: 0.47056
[660]	valid_0's multi_error: 0.470535
[661]	valid_0's multi_error: 0.470519
[662]	valid_0's multi_error: 0.470526
[663]	valid_0's multi_error: 0.470504
[664]	valid_0's multi_error: 0.470525
[665]	valid_0's multi_error: 0.470529
[666]	valid_0's multi_error: 0.470511
[667]	valid_0's multi_error: 0.470481
[668]	valid_0's multi_error: 0.470466
[669]	valid_0's multi_error: 0.470489
[670]	valid_0's multi_error: 0.470504
[671]	valid_0's multi_error: 0.470493
[672]	valid_0's multi_error: 0.470477
[673]	valid_0's multi_error: 0.470517
[674]	valid_0's multi_error: 0.470462
[675]	valid_0's multi_error: 0.470437
[676]	valid_0's multi_error: 0.470399
[677]	valid_0's multi_error: 0.470426
[678]	valid_0

[869]	valid_0's multi_error: 0.469025
[870]	valid_0's multi_error: 0.46902
[871]	valid_0's multi_error: 0.469005
[872]	valid_0's multi_error: 0.469013
[873]	valid_0's multi_error: 0.469017
[874]	valid_0's multi_error: 0.469013
[875]	valid_0's multi_error: 0.469037
[876]	valid_0's multi_error: 0.469034
[877]	valid_0's multi_error: 0.46904
[878]	valid_0's multi_error: 0.469026
[879]	valid_0's multi_error: 0.469002
[880]	valid_0's multi_error: 0.469053
[881]	valid_0's multi_error: 0.469025
[882]	valid_0's multi_error: 0.469031
[883]	valid_0's multi_error: 0.469031
[884]	valid_0's multi_error: 0.469022
[885]	valid_0's multi_error: 0.468998
[886]	valid_0's multi_error: 0.469008
[887]	valid_0's multi_error: 0.469007
[888]	valid_0's multi_error: 0.469037
[889]	valid_0's multi_error: 0.469001
[890]	valid_0's multi_error: 0.46899
[891]	valid_0's multi_error: 0.468981
[892]	valid_0's multi_error: 0.468958
[893]	valid_0's multi_error: 0.468944
[894]	valid_0's multi_error: 0.468998
[895]	valid_0's

...

[1505]	valid_0's multi_error: 0.466517
[1506]	valid_0's multi_error: 0.46649
[1507]	valid_0's multi_error: 0.466469
[1508]	valid_0's multi_error: 0.466468
[1509]	valid_0's multi_error: 0.466481
[1510]	valid_0's multi_error: 0.466487
[1511]	valid_0's multi_error: 0.466508
[1512]	valid_0's multi_error: 0.466481
[1513]	valid_0's multi_error: 0.466499
[1514]	valid_0's multi_error: 0.466513
[1515]	valid_0's multi_error: 0.466516
[1516]	valid_0's multi_error: 0.466525
[1517]	valid_0's multi_error: 0.466499
[1518]	valid_0's multi_error: 0.466492
[1519]	valid_0's multi_error: 0.466441
[1520]	valid_0's multi_error: 0.466441
[1521]	valid_0's multi_error: 0.46646
[1522]	valid_0's multi_error: 0.466499
[1523]	valid_0's multi_error: 0.466462
[1524]	valid_0's multi_error: 0.46646
[1525]	valid_0's multi_error: 0.466452
[1526]	valid_0's multi_error: 0.466456
[1527]	valid_0's multi_error: 0.466493
[1528]	valid_0's multi_error: 0.466468
[1529]	valid_0's multi_error: 0.466459
[1530]	valid_0's multi_error

[1716]	valid_0's multi_error: 0.466159
[1717]	valid_0's multi_error: 0.466165
[1718]	valid_0's multi_error: 0.466171
[1719]	valid_0's multi_error: 0.466174
[1720]	valid_0's multi_error: 0.466162
[1721]	valid_0's multi_error: 0.466137
[1722]	valid_0's multi_error: 0.466143
[1723]	valid_0's multi_error: 0.466146
[1724]	valid_0's multi_error: 0.46614
[1725]	valid_0's multi_error: 0.466087
[1726]	valid_0's multi_error: 0.466096
[1727]	valid_0's multi_error: 0.466072
[1728]	valid_0's multi_error: 0.466101
[1729]	valid_0's multi_error: 0.466129
[1730]	valid_0's multi_error: 0.46611
[1731]	valid_0's multi_error: 0.466135
[1732]	valid_0's multi_error: 0.466098
[1733]	valid_0's multi_error: 0.466102
[1734]	valid_0's multi_error: 0.466149
[1735]	valid_0's multi_error: 0.466134
[1736]	valid_0's multi_error: 0.466131
[1737]	valid_0's multi_error: 0.46612
[1738]	valid_0's multi_error: 0.466149
[1739]	valid_0's multi_error: 0.46612
[1740]	valid_0's multi_error: 0.46611
[1741]	valid_0's multi_error: 

[1927]	valid_0's multi_error: 0.465734
[1928]	valid_0's multi_error: 0.465713
[1929]	valid_0's multi_error: 0.465726
[1930]	valid_0's multi_error: 0.46574
[1931]	valid_0's multi_error: 0.465708
[1932]	valid_0's multi_error: 0.465728
[1933]	valid_0's multi_error: 0.465719
[1934]	valid_0's multi_error: 0.465722
[1935]	valid_0's multi_error: 0.465726
[1936]	valid_0's multi_error: 0.465708
[1937]	valid_0's multi_error: 0.465702
[1938]	valid_0's multi_error: 0.465707
[1939]	valid_0's multi_error: 0.465729
[1940]	valid_0's multi_error: 0.465732
[1941]	valid_0's multi_error: 0.465701
[1942]	valid_0's multi_error: 0.465716
[1943]	valid_0's multi_error: 0.465699
[1944]	valid_0's multi_error: 0.465681
[1945]	valid_0's multi_error: 0.465666
[1946]	valid_0's multi_error: 0.465665
[1947]	valid_0's multi_error: 0.465677
[1948]	valid_0's multi_error: 0.465675
[1949]	valid_0's multi_error: 0.465674
[1950]	valid_0's multi_error: 0.46562
[1951]	valid_0's multi_error: 0.465638
[1952]	valid_0's multi_erro

[2138]	valid_0's multi_error: 0.465556
[2139]	valid_0's multi_error: 0.465547
[2140]	valid_0's multi_error: 0.465556
[2141]	valid_0's multi_error: 0.46555
[2142]	valid_0's multi_error: 0.465563
[2143]	valid_0's multi_error: 0.465552
[2144]	valid_0's multi_error: 0.465532
[2145]	valid_0's multi_error: 0.465499
[2146]	valid_0's multi_error: 0.46551
[2147]	valid_0's multi_error: 0.465516
[2148]	valid_0's multi_error: 0.465525
[2149]	valid_0's multi_error: 0.465493
[2150]	valid_0's multi_error: 0.465516
[2151]	valid_0's multi_error: 0.465501
[2152]	valid_0's multi_error: 0.465486
[2153]	valid_0's multi_error: 0.46551
[2154]	valid_0's multi_error: 0.465492
[2155]	valid_0's multi_error: 0.465501
[2156]	valid_0's multi_error: 0.465504
[2157]	valid_0's multi_error: 0.465507
[2158]	valid_0's multi_error: 0.465496
[2159]	valid_0's multi_error: 0.465498
[2160]	valid_0's multi_error: 0.465492
[2161]	valid_0's multi_error: 0.465508
[2162]	valid_0's multi_error: 0.465508
[2163]	valid_0's multi_error

KeyboardInterrupt: 

In [ ]:
param = {
    'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 7,  
    'metric': {'multi_error', 'multi_logloss'},  
    'num_leaves': 500,
    'min_data_in_leaf': 200,  
    'learning_rate': 0.04,  
    'feature_fraction': 0.8,
    'lambda_l1': 0.4,  
    'lambda_l2': 0.2
}


X = trainFeatures.drop(['uId'], axis=1)
y = trainLabels
uid = testFeatures['uId']
test = testFeatures.drop('uId', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=2, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=10000, valid_sets=[vali], early_stopping_rounds=500)
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    y_test = model.predict(test,num_iteration=model.best_iteration)
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission-2.csv',index=False)

0
training start...
[1]	valid_0's multi_logloss: 1.59203	valid_0's multi_error: 0.701493
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's multi_logloss: 1.57517	valid_0's multi_error: 0.701493
[3]	valid_0's multi_logloss: 1.55955	valid_0's multi_error: 0.701474
[4]	valid_0's multi_logloss: 1.54587	valid_0's multi_error: 0.700117
[5]	valid_0's multi_logloss: 1.53278	valid_0's multi_error: 0.681419
[6]	valid_0's multi_logloss: 1.52006	valid_0's multi_error: 0.666591
[7]	valid_0's multi_logloss: 1.50784	valid_0's multi_error: 0.654442
[8]	valid_0's multi_logloss: 1.49629	valid_0's multi_error: 0.643195
[9]	valid_0's multi_logloss: 1.48534	valid_0's multi_error: 0.633214
[10]	valid_0's multi_logloss: 1.47517	valid_0's multi_error: 0.624164
[11]	valid_0's multi_logloss: 1.46541	valid_0's multi_error: 0.615706
[12]	valid_0's multi_logloss: 1.45622	valid_0's multi_error: 0.607185
[13]	valid_0's multi_logloss: 1.4477	valid_0's multi_error: 0.598987
[14]	valid_0's mu

In [8]:
gbm = lgb.LGBMClassifier()
params = {
    'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 7,  
    'metric': {'multi_error', 'multi_logloss'},  
    'max_depth': 15,
    'num_leaves': 200,
    'min_data_in_leaf': 200,  
    'learning_rate': 0.015,  
    'feature_fraction': 0.8,
    'lambda_l1': 0.4,  
    'lambda_l2': 0.2
}

train = lgb.Dataset(X_train, y_train)
valid = lgb.Dataset(X_test, y_test, reference=train)
gbm = lgb.train(params,  
                train,  
                num_boost_round=10000,  
                valid_sets=valid,
                early_stopping_rounds=500)

[1]	valid_0's multi_error: 0.702313	valid_0's multi_logloss: 1.60113
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's multi_error: 0.702313	valid_0's multi_logloss: 1.59151
[3]	valid_0's multi_error: 0.702313	valid_0's multi_logloss: 1.58158
[4]	valid_0's multi_error: 0.702313	valid_0's multi_logloss: 1.57296
[5]	valid_0's multi_error: 0.702313	valid_0's multi_logloss: 1.56407
[6]	valid_0's multi_error: 0.702313	valid_0's multi_logloss: 1.55507
[7]	valid_0's multi_error: 0.702313	valid_0's multi_logloss: 1.54713
[8]	valid_0's multi_error: 0.701935	valid_0's multi_logloss: 1.53856
[9]	valid_0's multi_error: 0.699965	valid_0's multi_logloss: 1.53043
[10]	valid_0's multi_error: 0.697677	valid_0's multi_logloss: 1.52284
[11]	valid_0's multi_error: 0.692945	valid_0's multi_logloss: 1.51493
[12]	valid_0's multi_error: 0.675756	valid_0's multi_logloss: 1.50723
[13]	valid_0's multi_error: 0.660353	valid_0's multi_logloss: 1.50005
[14]	valid_0's multi_error: 0.647706

[118]	valid_0's multi_error: 0.449741	valid_0's multi_logloss: 1.15708
[119]	valid_0's multi_error: 0.449592	valid_0's multi_logloss: 1.15563
[120]	valid_0's multi_error: 0.449303	valid_0's multi_logloss: 1.15424
[121]	valid_0's multi_error: 0.449085	valid_0's multi_logloss: 1.15288
[122]	valid_0's multi_error: 0.44894	valid_0's multi_logloss: 1.1515
[123]	valid_0's multi_error: 0.448826	valid_0's multi_logloss: 1.15011
[124]	valid_0's multi_error: 0.448627	valid_0's multi_logloss: 1.1488
[125]	valid_0's multi_error: 0.448517	valid_0's multi_logloss: 1.14747
[126]	valid_0's multi_error: 0.448398	valid_0's multi_logloss: 1.14615
[127]	valid_0's multi_error: 0.448249	valid_0's multi_logloss: 1.1449
[128]	valid_0's multi_error: 0.448015	valid_0's multi_logloss: 1.14363
[129]	valid_0's multi_error: 0.447811	valid_0's multi_logloss: 1.14233
[130]	valid_0's multi_error: 0.447726	valid_0's multi_logloss: 1.1411
[131]	valid_0's multi_error: 0.447423	valid_0's multi_logloss: 1.13988
[132]	valid

[234]	valid_0's multi_error: 0.437368	valid_0's multi_logloss: 1.06144
[235]	valid_0's multi_error: 0.437333	valid_0's multi_logloss: 1.06097
[236]	valid_0's multi_error: 0.437323	valid_0's multi_logloss: 1.0605
[237]	valid_0's multi_error: 0.437159	valid_0's multi_logloss: 1.06003
[238]	valid_0's multi_error: 0.437129	valid_0's multi_logloss: 1.05961
[239]	valid_0's multi_error: 0.437119	valid_0's multi_logloss: 1.05919
[240]	valid_0's multi_error: 0.43702	valid_0's multi_logloss: 1.05876
[241]	valid_0's multi_error: 0.43694	valid_0's multi_logloss: 1.05832
[242]	valid_0's multi_error: 0.436891	valid_0's multi_logloss: 1.05787
[243]	valid_0's multi_error: 0.436826	valid_0's multi_logloss: 1.05742
[244]	valid_0's multi_error: 0.436706	valid_0's multi_logloss: 1.05697
[245]	valid_0's multi_error: 0.436567	valid_0's multi_logloss: 1.05653
[246]	valid_0's multi_error: 0.436537	valid_0's multi_logloss: 1.05609
[247]	valid_0's multi_error: 0.436483	valid_0's multi_logloss: 1.05566
[248]	val

[350]	valid_0's multi_error: 0.429443	valid_0's multi_logloss: 1.02395
[351]	valid_0's multi_error: 0.429358	valid_0's multi_logloss: 1.02372
[352]	valid_0's multi_error: 0.429318	valid_0's multi_logloss: 1.02351
[353]	valid_0's multi_error: 0.429294	valid_0's multi_logloss: 1.02329
[354]	valid_0's multi_error: 0.429109	valid_0's multi_logloss: 1.02306
[355]	valid_0's multi_error: 0.429065	valid_0's multi_logloss: 1.02285
[356]	valid_0's multi_error: 0.42902	valid_0's multi_logloss: 1.02263
[357]	valid_0's multi_error: 0.42899	valid_0's multi_logloss: 1.02242
[358]	valid_0's multi_error: 0.429005	valid_0's multi_logloss: 1.02221
[359]	valid_0's multi_error: 0.42897	valid_0's multi_logloss: 1.022
[360]	valid_0's multi_error: 0.428871	valid_0's multi_logloss: 1.02178
[361]	valid_0's multi_error: 0.428856	valid_0's multi_logloss: 1.02156
[362]	valid_0's multi_error: 0.428821	valid_0's multi_logloss: 1.02135
[363]	valid_0's multi_error: 0.428736	valid_0's multi_logloss: 1.02114
[364]	valid

[466]	valid_0's multi_error: 0.423458	valid_0's multi_logloss: 1.00398
[467]	valid_0's multi_error: 0.423403	valid_0's multi_logloss: 1.00385
[468]	valid_0's multi_error: 0.423383	valid_0's multi_logloss: 1.00372
[469]	valid_0's multi_error: 0.423323	valid_0's multi_logloss: 1.00359
[470]	valid_0's multi_error: 0.423259	valid_0's multi_logloss: 1.00345
[471]	valid_0's multi_error: 0.423134	valid_0's multi_logloss: 1.00333
[472]	valid_0's multi_error: 0.4231	valid_0's multi_logloss: 1.0032
[473]	valid_0's multi_error: 0.42308	valid_0's multi_logloss: 1.00306
[474]	valid_0's multi_error: 0.42307	valid_0's multi_logloss: 1.00293
[475]	valid_0's multi_error: 0.422995	valid_0's multi_logloss: 1.0028
[476]	valid_0's multi_error: 0.422995	valid_0's multi_logloss: 1.00268
[477]	valid_0's multi_error: 0.42294	valid_0's multi_logloss: 1.00255
[478]	valid_0's multi_error: 0.42294	valid_0's multi_logloss: 1.00242
[479]	valid_0's multi_error: 0.422841	valid_0's multi_logloss: 1.00228
[480]	valid_0'

...

[5564]	valid_0's multi_error: 0.39399	valid_0's multi_logloss: 0.929945
[5565]	valid_0's multi_error: 0.39396	valid_0's multi_logloss: 0.929942
[5566]	valid_0's multi_error: 0.39394	valid_0's multi_logloss: 0.929942
[5567]	valid_0's multi_error: 0.393945	valid_0's multi_logloss: 0.929939
[5568]	valid_0's multi_error: 0.39395	valid_0's multi_logloss: 0.929938
[5569]	valid_0's multi_error: 0.393985	valid_0's multi_logloss: 0.929936
[5570]	valid_0's multi_error: 0.39398	valid_0's multi_logloss: 0.929937
[5571]	valid_0's multi_error: 0.393955	valid_0's multi_logloss: 0.929938
[5572]	valid_0's multi_error: 0.393985	valid_0's multi_logloss: 0.929936
[5573]	valid_0's multi_error: 0.39398	valid_0's multi_logloss: 0.929937
[5574]	valid_0's multi_error: 0.393975	valid_0's multi_logloss: 0.929934
[5575]	valid_0's multi_error: 0.39397	valid_0's multi_logloss: 0.929931
[5576]	valid_0's multi_error: 0.393995	valid_0's multi_logloss: 0.929932
[5577]	valid_0's multi_error: 0.393985	valid_0's multi_log

[5678]	valid_0's multi_error: 0.393915	valid_0's multi_logloss: 0.929819
[5679]	valid_0's multi_error: 0.39398	valid_0's multi_logloss: 0.929817
[5680]	valid_0's multi_error: 0.393965	valid_0's multi_logloss: 0.929815
[5681]	valid_0's multi_error: 0.39394	valid_0's multi_logloss: 0.929813
[5682]	valid_0's multi_error: 0.39396	valid_0's multi_logloss: 0.929813
[5683]	valid_0's multi_error: 0.393955	valid_0's multi_logloss: 0.929812
[5684]	valid_0's multi_error: 0.393985	valid_0's multi_logloss: 0.929811
[5685]	valid_0's multi_error: 0.39399	valid_0's multi_logloss: 0.929809
[5686]	valid_0's multi_error: 0.393995	valid_0's multi_logloss: 0.929809
[5687]	valid_0's multi_error: 0.39402	valid_0's multi_logloss: 0.929809
[5688]	valid_0's multi_error: 0.39397	valid_0's multi_logloss: 0.929805
[5689]	valid_0's multi_error: 0.39397	valid_0's multi_logloss: 0.929804
[5690]	valid_0's multi_error: 0.39397	valid_0's multi_logloss: 0.929804
[5691]	valid_0's multi_error: 0.39397	valid_0's multi_loglo

[5792]	valid_0's multi_error: 0.394234	valid_0's multi_logloss: 0.929683
[5793]	valid_0's multi_error: 0.394259	valid_0's multi_logloss: 0.929682
[5794]	valid_0's multi_error: 0.394169	valid_0's multi_logloss: 0.929681
[5795]	valid_0's multi_error: 0.394154	valid_0's multi_logloss: 0.929678
[5796]	valid_0's multi_error: 0.394189	valid_0's multi_logloss: 0.929679
[5797]	valid_0's multi_error: 0.394204	valid_0's multi_logloss: 0.929678
[5798]	valid_0's multi_error: 0.394214	valid_0's multi_logloss: 0.929674
[5799]	valid_0's multi_error: 0.394209	valid_0's multi_logloss: 0.929672
[5800]	valid_0's multi_error: 0.394209	valid_0's multi_logloss: 0.929672
[5801]	valid_0's multi_error: 0.394169	valid_0's multi_logloss: 0.929672
[5802]	valid_0's multi_error: 0.394139	valid_0's multi_logloss: 0.92967
[5803]	valid_0's multi_error: 0.394174	valid_0's multi_logloss: 0.929668
[5804]	valid_0's multi_error: 0.394164	valid_0's multi_logloss: 0.929669
[5805]	valid_0's multi_error: 0.394124	valid_0's mul

[5905]	valid_0's multi_error: 0.39409	valid_0's multi_logloss: 0.929569
[5906]	valid_0's multi_error: 0.394104	valid_0's multi_logloss: 0.929567
[5907]	valid_0's multi_error: 0.394114	valid_0's multi_logloss: 0.929565
[5908]	valid_0's multi_error: 0.394085	valid_0's multi_logloss: 0.929562
[5909]	valid_0's multi_error: 0.39409	valid_0's multi_logloss: 0.929561
[5910]	valid_0's multi_error: 0.39408	valid_0's multi_logloss: 0.929559
[5911]	valid_0's multi_error: 0.394085	valid_0's multi_logloss: 0.929557
[5912]	valid_0's multi_error: 0.39409	valid_0's multi_logloss: 0.929555
[5913]	valid_0's multi_error: 0.394095	valid_0's multi_logloss: 0.929553
[5914]	valid_0's multi_error: 0.39408	valid_0's multi_logloss: 0.929554
[5915]	valid_0's multi_error: 0.394095	valid_0's multi_logloss: 0.929553
[5916]	valid_0's multi_error: 0.394109	valid_0's multi_logloss: 0.929549
[5917]	valid_0's multi_error: 0.39407	valid_0's multi_logloss: 0.929549
[5918]	valid_0's multi_error: 0.394045	valid_0's multi_lo

[6019]	valid_0's multi_error: 0.393891	valid_0's multi_logloss: 0.929446
[6020]	valid_0's multi_error: 0.39392	valid_0's multi_logloss: 0.929445
[6021]	valid_0's multi_error: 0.393935	valid_0's multi_logloss: 0.929444
[6022]	valid_0's multi_error: 0.39395	valid_0's multi_logloss: 0.929442
[6023]	valid_0's multi_error: 0.39397	valid_0's multi_logloss: 0.92944
[6024]	valid_0's multi_error: 0.393945	valid_0's multi_logloss: 0.929441
[6025]	valid_0's multi_error: 0.393955	valid_0's multi_logloss: 0.92944
[6026]	valid_0's multi_error: 0.39396	valid_0's multi_logloss: 0.929438
[6027]	valid_0's multi_error: 0.39397	valid_0's multi_logloss: 0.929438
[6028]	valid_0's multi_error: 0.39396	valid_0's multi_logloss: 0.929437
[6029]	valid_0's multi_error: 0.393955	valid_0's multi_logloss: 0.929436
[6030]	valid_0's multi_error: 0.393945	valid_0's multi_logloss: 0.929436
[6031]	valid_0's multi_error: 0.39395	valid_0's multi_logloss: 0.929435
[6032]	valid_0's multi_error: 0.39393	valid_0's multi_loglos

In [9]:
out = gbm.predict(testFeatures)
outres=[list(x).index(max(x)) for x in out]
result = pd.DataFrame({'id':testFeatures.uId, 'label':outres})
result.to_csv("NewResul54.csv", index=False)

In [10]:
out = gbm.predict(testFeatures)
#outres=[list(x).index(max(x)) for x in out]
result = pd.DataFrame({'id':testFeatures.uId, 'p':list(out)})
result.to_csv("lgbm-pro.csv", index=False)